<a href="https://colab.research.google.com/github/botingchen/ML_Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis

api_key = "c9d26a6782e25df032d62899e2bbe7ef"
# https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key=<<api_key>>&language=en-US
response = requests.get('https://api.themoviedb.org/3/movie/181808/credits?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')

In [82]:
highest_revenue = response.json()
for i in highest_revenue['crew']:
  print(i)
print(highest_revenue)

{'adult': False, 'gender': 2, 'id': 1, 'known_for_department': 'Directing', 'name': 'George Lucas', 'original_name': 'George Lucas', 'popularity': 9.198, 'profile_path': '/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg', 'credit_id': '52fe4c709251416c7511a87b', 'department': 'Writing', 'job': 'Characters'}
{'adult': False, 'gender': 1, 'id': 4, 'known_for_department': 'Acting', 'name': 'Carrie Fisher', 'original_name': 'Carrie Fisher', 'popularity': 8.889, 'profile_path': '/rfJtncHewKVnHjqpIZvjn24ESeC.jpg', 'credit_id': '5a398f5dc3a36814ae07f732', 'department': 'Crew', 'job': 'In Memory Of'}
{'adult': False, 'gender': 2, 'id': 491, 'known_for_department': 'Sound', 'name': 'John Williams', 'original_name': 'John Williams', 'popularity': 3.157, 'profile_path': '/KFyMqUWeiBdP9tJcZyGWOqnrgK.jpg', 'credit_id': '575bcbb1925141649b000fbb', 'department': 'Sound', 'job': 'Original Music Composer'}
{'adult': False, 'gender': 2, 'id': 1226, 'known_for_department': 'Art', 'name': 'Rick Heinrichs', 'original_name'